In [1]:
using Glob
using Dates
using DIVAnd
using NCDatasets
using GeoDatasets
using CairoMakie, GeoMakie
using JupyterFormatter
enable_autoformat()
include("./config.jl")
monthlist = [Dates.monthname(mm) for mm = 1:12];

## All regions together

In [173]:
for varname in varlist
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    obsyears = Int64[]
    obsmonths = Int64[]

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsvalue, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)
        append!(obsyears, Dates.year.(obstime))
        append!(obsmonths, Dates.month.(obstime))
    end

    @info(extrema(obsyears))
    yearmin = minimum(obsyears)
    yearmax = maximum(obsyears)
    nobs_month = [length(findall(obsmonths .== mm)) for mm = 1:12]
    nobs_year = [length(findall(obsyears .== yyyy)) for yyyy = yearmin:yearmax]

    # Make figure
    fig1 = make_histogram(yearmin, yearmax, nobs_year, nobs_month, varname_)
    save(joinpath(figdir, "histogram_$varname.png"), fig1)
end

[ Info: (1960, 2023)
[ Info: (1963, 2024)
[ Info: (1960, 2023)
[ Info: (1950, 2023)
[ Info: (1928, 2023)
[ Info: (1902, 2024)


## One color by region

In [63]:
yearmin = 1960
yearmax = 2024

for varname in varlist
    @info("Working on variable $(varname)")
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    nyears_region = zeros(length(datafilelist), yearmax - yearmin + 1)
    nmonths_region = zeros(length(datafilelist), 12)

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsvalue, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)

        obsmonths = Dates.month.(obstime)
        obsyears = Dates.year.(obstime)
        nmonths_region[iii, :] = [length(findall(obsmonths .== mm)) for mm = 1:12]
        nyears_region[iii, :] =
            [length(findall(obsyears .== yyyy)) for yyyy = yearmin:yearmax]
    end

    fig = Figure(size = (1500, 500))
    ax = Axis(
        fig[1, 1],
        title = "Number of $(varname) observations per year",
        xticks = 1960:10:2020,
    )

    ax2 = Axis(
        fig[1, 2],
        xticks = 1:12,
        xtickformat = x -> Dates.monthname.(Int.(x)),
        xticklabelrotation = pi / 6,
        title = "Number of $(varname) observations per month",
    )

    bplist = []
    for jjj = 6:-1:1
        bp = barplot!(
            ax,
            yearmin:yearmax,
            dropdims(sum(nyears_region[1:jjj, :], dims = 1), dims = 1),
            strokewidth = 0.5,
            strokecolor = (:black, 0.5),
            color = collect(values(domaincolors))[jjj],
            label = collect(keys(domaincolors))[jjj],
        )
        push!(bplist, bp)
        barplot!(
            ax2,
            1:12,
            dropdims(sum(nmonths_region[1:jjj, :], dims = 1), dims = 1),
            strokewidth = 0.5,
            strokecolor = (:black, 0.5),
            color = collect(values(domaincolors))[jjj],
        )
        hidespines!(ax2, :t, :r)
    end
    axislegend(ax, reverse(bplist), collect(keys(domaincolors)), position = :lt)
    xlims!(ax, 1960 - 0.5, 2024 + 0.5)
    xlims!(ax2, 0.5, 12 + 0.5)
    hidespines!(ax, :t, :r)
    hidespines!(ax2, :t, :r)

    save(joinpath(figdir, "stacked_histogram_$varname.png"), fig)
end

[ Info: Working on variable Water_body_ammonium
[ Info: Working on variable Water_body_chlorophyll-a
[ Info: Working on variable Water_body_dissolved_inorganic_nitrogen
[ Info: Working on variable Water_body_silicate
[ Info: Working on variable Water_body_phosphate
[ Info: Working on variable Water_body_dissolved_oxygen_concentration
